# A minimal ALF run

In this bare-bones example we use the [pyALF](https://github.com/ALF-QMC/pyALF) interface to run the canonical Hubbard model on a default configuration: a $6\times6$ square grid, with interaction strength $U=4$ and inverse temperature $\beta = 5$.  

Bellow we go through the steps for performing the simulation and outputting observables.

---

**1.** Import `ALF_source` and `Simulation` classes from the `py_alf` python module, which provide the interface with ALF:

In [1]:
from py_alf import ALF_source, Simulation  # Interface with ALF

**2.** Create an instance of `ALF_source`, downloading the ALF source code from the [ALF repository](https://github.com/ALF-QMC/ALF/-/tree/master/), if `alf_dir` does not exist. Gets alf_dir from environment variable `$ALF_DIR`, or defaults to "./ALF", if not present:

In [2]:
alf_src = ALF_source(branch='master')

Checking out branch master
Your branch is up to date with 'origin/master'.


Switched to branch 'master'


**3.** Create an instance of `Simulation`, overwriting default parameters as desired:

In [3]:
sim = Simulation(
    alf_src,
    "Hubbard",                    # Name of Hamiltonian
    {                             # Dictionary overwriting default parameters
        "Lattice_type": "Square"
    },
    machine='GNU'  # Change to "intel", or "PGI" if gfortran is not installed
)

**4.** Compile ALF. The first time it will also download and compile HDF5, which could take $\sim15$ minutes.

In [4]:
sim.compile()

Compiling ALF... 
Cleaning up Prog/
Cleaning up Libraries/
Cleaning up Analysis/
Compiling Libraries


entanglement_mod.F90:35:2:

   35 | #warning "You are compiling entanglement without MPI. No results possible"
      |  1~~~~~~
ar: creating modules_90.a
ar: creating libqrref.a


Compiling Analysis
Compiling Program
Parsing Hamiltonian parameters
filename: Hamiltonians/Hamiltonian_Kondo_smod.F90
filename: Hamiltonians/Hamiltonian_Hubbard_smod.F90
filename: Hamiltonians/Hamiltonian_Hubbard_Plain_Vanilla_smod.F90
filename: Hamiltonians/Hamiltonian_tV_smod.F90
filename: Hamiltonians/Hamiltonian_LRC_smod.F90
filename: Hamiltonians/Hamiltonian_Z2_Matter_smod.F90
Compiling program modules
Link program
Done.


**5.** Perform the simulation as specified in `sim`:

In [5]:
sim.run()

Prepare directory "/home/jonas/Programs/pyALF/Notebooks/ALF_data/Hubbard_Square" for Monte Carlo run.
Create new directory.
Run /home/jonas/Programs/ALF/Prog/ALF.out
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration


**6.** Perform some simple analysis:

In [6]:
sim.analysis()

### Analyzing /home/jonas/Programs/pyALF/Notebooks/ALF_data/Hubbard_Square ###
/home/jonas/Programs/pyALF/Notebooks
Scalar observables:
Ener_scal
Kin_scal
Part_scal
Pot_scal
Histogram observables:
Equal time observables:
Den_eq
Green_eq
SpinT_eq
SpinXY_eq
SpinZ_eq
Time displaced observables:
Den_tau
Green_tau
SpinT_tau
SpinXY_tau
SpinZ_tau


**7.** Read analysis results into a Pandas Dataframe with one row per simulation, containing parameters and observables:

In [7]:
obs = sim.get_obs()

/home/jonas/Programs/pyALF/Notebooks/ALF_data/Hubbard_Square
No orbital locations saved.


In [8]:
obs

,continuous,ham_chem,ham_t,ham_t2,ham_tperp,ham_u,ham_u2,mz,l1,l2,...,SpinXY_tauK_err,SpinXY_tauR,SpinXY_tauR_err,SpinXY_tau_lattice,SpinZ_tauK,SpinZ_tauK_err,SpinZ_tauR,SpinZ_tauR_err,SpinZ_tau_lattice,lattice
/home/jonas/Programs/pyALF/Notebooks/ALF_data/Hubbard_Square,0,0.0,1.0,1.0,1.0,4.0,4.0,1,6,6,...,"[[0.22628722504550844, 0.37685459854025805, 0....","[[0.0575988003757706, -0.10378742200171556, 0....","[[0.01248834666799511, 0.036871289378260834, 0...","{'L1': [6.0, 0.0], 'L2': [0.0, 6.0], 'a1': [1....","[[0.9040036362034092, 0.5628191896020613, 0.61...","[[0.1494786750311893, 0.06006336638595281, 0.0...","[[0.1013442053139478, -0.11445552391592478, 0....","[[0.06273438448505823, 0.056390780430637014, 0...","{'L1': [6.0, 0.0], 'L2': [0.0, 6.0], 'a1': [1....","{'L1': [6.0, 0.0], 'L2': [0.0, 6.0], 'N_coord'..."


$\bullet$ The internal energy of the system (and its error) are accessed by:

In [9]:
obs.iloc[0][['Ener_scal0', 'Ener_scal0_err', 'Ener_scal_sign', 'Ener_scal_sign_err']]

Ener_scal0           -29.821914
Ener_scal0_err          0.13032
Ener_scal_sign              1.0
Ener_scal_sign_err          0.0
Name: /home/jonas/Programs/pyALF/Notebooks/ALF_data/Hubbard_Square, dtype: object

$\bullet$ The simulation can be resumed by calling `sim.run()` again, increasing the precision of results:

In [10]:
sim.run()
sim.analysis()
obs2 = sim.get_obs()
obs2.iloc[0][['Ener_scal0', 'Ener_scal0_err', 'Ener_scal_sign', 'Ener_scal_sign_err']]

Prepare directory "/home/jonas/Programs/pyALF/Notebooks/ALF_data/Hubbard_Square" for Monte Carlo run.
Resuming previous run.
Run /home/jonas/Programs/ALF/Prog/ALF.out
 ALF Copyright (C) 2016 - 2021 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
### Analyzing /home/jonas/Programs/pyALF/Notebooks/ALF_data/Hubbard_Square ###
/home/jonas/Programs/pyALF/Notebooks
Scalar observables:
Ener_scal
Kin_scal
Part_scal
Pot_scal
Histogram observables:
Equal time observables:
Den_eq
Green_eq
SpinT_eq
SpinXY_eq
SpinZ_eq
Time displaced observables:
Den_tau
Green_tau
SpinT_tau
SpinXY_tau
SpinZ_tau
/home/jonas/Programs/pyALF/Notebooks/ALF_data/Hubbard_Square
No orbital locations saved.


Ener_scal0           -29.609245
Ener_scal0_err         0.136803
Ener_scal_sign              1.0
Ener_scal_sign_err          0.0
Name: /home/jonas/Programs/pyALF/Notebooks/ALF_data/Hubbard_Square, dtype: object

In [11]:
print("\nRunning again reduced the error from\n",
      obs.iloc[0][['Ener_scal0_err']], "\nto\n",
      obs2.iloc[0][['Ener_scal0_err']])


Running again reduced the error from
 Ener_scal0_err    0.13032
Name: /home/jonas/Programs/pyALF/Notebooks/ALF_data/Hubbard_Square, dtype: object 
to
 Ener_scal0_err    0.136803
Name: /home/jonas/Programs/pyALF/Notebooks/ALF_data/Hubbard_Square, dtype: object


**Note**: To run a fresh simulation - instead of performing a refinement over previous run(s) - the Monte Carlo run directory should be deleted before rerunning.

---
## Exercises
1. Check info/help commands such as `help(alf_src)`, `alf_src.get_ham_names()`, and `alf_src.get_default_params('Hubbard')`.
2. Rerun once again and check the new improvement in precision.
3. Look at a few other observables (`sim.analysis()` outputs the names of those available).
4. Change the lattice size by adding, e.g., `\"L1\": 4,` and `\"L2\": 1,` to the simulation parameters definitions of `sim` (step 2).